Intermodal nx_graph can be loaded from OSM [example](https://github.com/DDonnyy/IduEdu/blob/main/examples/get_any_graph.ipynb)

In [ ]:
from shapely import Point

from objectnat import get_intermodal_graph,get_accessibility_isochrones
import geopandas as gpd

G_intermodal = get_intermodal_graph(osm_id=1114252)

In [ ]:
points = gpd.GeoDataFrame(geometry=[Point(30.295606,59.9439234)],crs=4326).to_crs(G_intermodal.graph['crs'])
isochrones, stops, routes = get_accessibility_isochrones(
    points=points,
    weight_type="time_min",
    weight_value=10,
    graph_nx=G_intermodal
)
m = isochrones.explore(tiles='CartoDB positron')
stops.explore(m=m)
routes.explore(m=m, column='type')